In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [15]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [16]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

### Encoding categorical data

Label Encoding the "Gender" column

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [18]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

One Hot Encoding the "Geography" column

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

### Splitting the dataset into the Training set and Test set

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### Feature Scaling

In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN with k-cross validation

In [22]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [23]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=6, activation='relu'))
#     classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units=6, activation='relu'))
#     classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units=1, activation='sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics= ['accuracy'])
    return classifier

In [24]:
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [20],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                          param_grid = parameters,
                          scoring = 'accuracy',
                          cv = 10
                          )

C:\Users\sshuv\AppData\Local\Temp\ipykernel_1460\1693862911.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier)


In [25]:
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/20
288/288 [==============================] - 1s 2ms/step - loss: 0.5242 - accuracy: 0.7861
Epoch 2/20
288/288 [==============================] - 1s 2ms/step - loss: 0.4547 - accuracy: 0.7976
Epoch 3/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4368 - accuracy: 0.8071
Epoch 4/20
288/288 [==============================] - 1s 2ms/step - loss: 0.4274 - accuracy: 0.8192
Epoch 5/20
288/288 [==============================] - 1s 2ms/step - loss: 0.4210 - accuracy: 0.8211
Epoch 6/20
288/288 [==============================] - 1s 2ms/step - loss: 0.4152 - accuracy: 0.8238
Epoch 7/20
288/288 [==============================] - 1s 2ms/step - loss: 0.4100 - accuracy: 0.8247
Epoch 8/20
288/288 [==============================] - 1s 2ms/step - loss: 0.4057 - accuracy: 0.8267
Epoch 9/20
288/288 [==============================] - 1s 2ms/step - loss: 0.4014 - accuracy: 0.8267
Epoch 10/20
288/288 [==============================] - 1s 2ms/step - loss: 0.3972 - accuracy: 0.8282

25/25 [==============================] - 0s 1ms/step
Epoch 1/20
288/288 [==============================] - 1s 2ms/step - loss: 0.6494 - accuracy: 0.6263
Epoch 2/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4789 - accuracy: 0.7983
Epoch 3/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4487 - accuracy: 0.8072
Epoch 4/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4372 - accuracy: 0.8140
Epoch 5/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4291 - accuracy: 0.8217
Epoch 6/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4218 - accuracy: 0.8272
Epoch 7/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4145 - accuracy: 0.8308
Epoch 8/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4078 - accuracy: 0.8307
Epoch 9/20
288/288 [==============================] - 0s 2ms/step - loss: 0.3993 - accuracy: 0.8340
Epoch 10/20
288/288 [==========================

288/288 [==============================] - 0s 2ms/step - loss: 0.3360 - accuracy: 0.8637
Epoch 20/20
25/25 [==============================] - 0s 1ms/step
Epoch 1/20
288/288 [==============================] - 1s 2ms/step - loss: 0.5094 - accuracy: 0.7946
Epoch 2/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4571 - accuracy: 0.7946
Epoch 3/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4413 - accuracy: 0.7946
Epoch 4/20
288/288 [==============================] - 1s 2ms/step - loss: 0.4327 - accuracy: 0.7946
Epoch 5/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4273 - accuracy: 0.7958
Epoch 6/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4227 - accuracy: 0.8132
Epoch 7/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4186 - accuracy: 0.8138
Epoch 8/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4149 - accuracy: 0.8167
Epoch 9/20
288/288 [==========================

288/288 [==============================] - 0s 2ms/step - loss: 0.3706 - accuracy: 0.8444
Epoch 19/20
288/288 [==============================] - 0s 2ms/step - loss: 0.3688 - accuracy: 0.8467
Epoch 20/20
25/25 [==============================] - 0s 1ms/step
Epoch 1/20
288/288 [==============================] - 1s 2ms/step - loss: 0.5469 - accuracy: 0.7779
Epoch 2/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4843 - accuracy: 0.8000
Epoch 3/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4590 - accuracy: 0.8037
Epoch 4/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4435 - accuracy: 0.8062
Epoch 5/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4333 - accuracy: 0.8089
Epoch 6/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4259 - accuracy: 0.8125
Epoch 7/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4198 - accuracy: 0.8190
Epoch 8/20
288/288 [=========================

288/288 [==============================] - 0s 2ms/step - loss: 0.3998 - accuracy: 0.8318
Epoch 18/20
288/288 [==============================] - 0s 2ms/step - loss: 0.3970 - accuracy: 0.8322
Epoch 19/20
288/288 [==============================] - 0s 2ms/step - loss: 0.3946 - accuracy: 0.8335
Epoch 20/20
25/25 [==============================] - 0s 1ms/step
Epoch 1/20
288/288 [==============================] - 1s 2ms/step - loss: 0.5969 - accuracy: 0.7075
Epoch 2/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4856 - accuracy: 0.7928
Epoch 3/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4579 - accuracy: 0.7949
Epoch 4/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4444 - accuracy: 0.7974
Epoch 5/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4354 - accuracy: 0.8036
Epoch 6/20
288/288 [==============================] - 0s 2ms/step - loss: 0.4274 - accuracy: 0.8062
Epoch 7/20
288/288 [========================

...

225/225 [==============================] - 0s 2ms/step - loss: 0.3573 - accuracy: 0.8546
Epoch 16/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3557 - accuracy: 0.8592
Epoch 17/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3539 - accuracy: 0.8581
Epoch 18/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3528 - accuracy: 0.8589
Epoch 19/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3513 - accuracy: 0.8599
Epoch 20/20
25/25 [==============================] - 0s 1ms/step
Epoch 1/20
225/225 [==============================] - 1s 2ms/step - loss: 0.5526 - accuracy: 0.7525
Epoch 2/20
225/225 [==============================] - 0s 2ms/step - loss: 0.4671 - accuracy: 0.8046
Epoch 3/20
225/225 [==============================] - 0s 2ms/step - loss: 0.4466 - accuracy: 0.8126
Epoch 4/20
225/225 [==============================] - 0s 2ms/step - loss: 0.4345 - accuracy: 0.8151
Epoch 5/20
225/225 [======================

225/225 [==============================] - 0s 2ms/step - loss: 0.3703 - accuracy: 0.8217
Epoch 15/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3668 - accuracy: 0.8222
Epoch 16/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3644 - accuracy: 0.8228
Epoch 17/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3624 - accuracy: 0.8226
Epoch 18/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3614 - accuracy: 0.8232
Epoch 19/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3602 - accuracy: 0.8257
Epoch 20/20
25/25 [==============================] - 0s 1ms/step
Epoch 1/20
225/225 [==============================] - 1s 2ms/step - loss: 0.5687 - accuracy: 0.7131
Epoch 2/20
225/225 [==============================] - 0s 2ms/step - loss: 0.4609 - accuracy: 0.7989
Epoch 3/20
225/225 [==============================] - 0s 2ms/step - loss: 0.4419 - accuracy: 0.8087
Epoch 4/20
225/225 [=====================

225/225 [==============================] - 0s 2ms/step - loss: 0.3944 - accuracy: 0.8364
Epoch 14/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3886 - accuracy: 0.8404
Epoch 15/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3817 - accuracy: 0.8436
Epoch 16/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3755 - accuracy: 0.8475
Epoch 17/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3696 - accuracy: 0.8526
Epoch 18/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3646 - accuracy: 0.8542
Epoch 19/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3601 - accuracy: 0.8568
Epoch 20/20
25/25 [==============================] - 0s 1ms/step
Epoch 1/20
225/225 [==============================] - 1s 2ms/step - loss: 0.5360 - accuracy: 0.7788
Epoch 2/20
225/225 [==============================] - 0s 2ms/step - loss: 0.4567 - accuracy: 0.7968
Epoch 3/20
225/225 [====================

225/225 [==============================] - 0s 2ms/step - loss: 0.3648 - accuracy: 0.8518
Epoch 13/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3620 - accuracy: 0.8535
Epoch 14/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3603 - accuracy: 0.8531
Epoch 15/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3588 - accuracy: 0.8544
Epoch 16/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3579 - accuracy: 0.8540
Epoch 17/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3565 - accuracy: 0.8540
Epoch 18/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3561 - accuracy: 0.8565
Epoch 19/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3554 - accuracy: 0.8553
Epoch 20/20
25/25 [==============================] - 0s 1ms/step
Epoch 1/20
225/225 [==============================] - 1s 2ms/step - loss: 0.6031 - accuracy: 0.6776
Epoch 2/20
225/225 [===================

225/225 [==============================] - 0s 2ms/step - loss: 0.3749 - accuracy: 0.8446
Epoch 12/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3663 - accuracy: 0.8479
Epoch 13/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3605 - accuracy: 0.8504
Epoch 14/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3566 - accuracy: 0.8506
Epoch 15/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3538 - accuracy: 0.8539
Epoch 16/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3518 - accuracy: 0.8537
Epoch 17/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3506 - accuracy: 0.8567
Epoch 18/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3490 - accuracy: 0.8560
Epoch 19/20
225/225 [==============================] - 0s 2ms/step - loss: 0.3484 - accuracy: 0.8558
Epoch 20/20
25/25 [==============================] - 0s 1ms/step
Epoch 1/20
250/250 [==================

In [26]:
print(best_parameters, best_accuracy)

{'batch_size': 32, 'epochs': 20, 'optimizer': 'adam'} 0.851375
